In [1]:
import yfinance as yf
import pandas as pd

In [2]:
hk_tickers = ['2800.HK']

# Define years you're interested in
years = [2021, 2022, 2023, 2024]

# Dictionary to store results
results = {}

for ticker in hk_tickers:
    try:
        # Download historical data
        data = yf.download(ticker, start="2021-01-01", end='2025-01-01')

        yearly_data = {}

        for year in years:
            yearly_subset = data[data.index.year == year]
            if not yearly_subset.empty:
                # Extract scalar float values using .item() or direct access
                first_close = yearly_subset.iloc[0]['Close'].item()
                last_close = yearly_subset.iloc[-1]['Close'].item()
                yearly_data[year] = {
                    'First Trading Day Close': round(first_close, 5),
                    'Last Trading Day Close': round(last_close, 5)
                }
            else:
                yearly_data[year] = {
                    'First Trading Day Close': None,
                    'Last Trading Day Close': None
                }

        results[ticker] = yearly_data

    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        results[ticker] = {year: {'First Trading Day Close': None, 'Last Trading Day Close': None} for year in years}

# Build DataFrame correctly from results
df_list = []

for ticker, data in results.items():
    for year, values in data.items():
        row = {
            'Ticker': ticker,
            'Year': year,
            'First_Close': values['First Trading Day Close'],
            'Last_Close': values['Last Trading Day Close']
        }
        df_list.append(row)

df = pd.DataFrame(df_list)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


In [3]:
df

,Ticker,Year,First_Close,Last_Close
0,2800.HK,2021,23.82997,20.79711
1,2800.HK,2022,20.69100,18.32262
2,2800.HK,2023,18.63535,16.38940
3,2800.HK,2024,16.15105,20.04050


In [4]:
df['Difference'] = df['Last_Close'] - df['First_Close']
df

,Ticker,Year,First_Close,Last_Close,Difference
0,2800.HK,2021,23.82997,20.79711,-3.03286
1,2800.HK,2022,20.69100,18.32262,-2.36838
2,2800.HK,2023,18.63535,16.38940,-2.24595
3,2800.HK,2024,16.15105,20.04050,3.88945


In [5]:
# Create an empty list to store dividend data
dividend_data = []

# Loop through each ticker
for ticker in hk_tickers:
    stock = yf.Ticker(ticker)
    dividends = stock.dividends

    if not dividends.empty:
        # Convert index to year
        dividends = dividends.to_frame(name='Dividends')
        dividends['Year'] = dividends.index.year
        dividends['Ticker'] = ticker

        # Filter by custom year range
        mask = (dividends['Year'] >= 2021) & (dividends['Year'] <= 2024)
        filtered_dividends = dividends.loc[mask]

        # Group by Year and sum the dividends
        yearly_div = filtered_dividends.groupby(['Ticker', 'Year'])['Dividends'].sum().reset_index()

        # Append to final list
        dividend_data.append(yearly_div)

# Concatenate all ticker data into one DataFrame
df_div = pd.concat(dividend_data, ignore_index=True)

# Display the result
df_div

,Ticker,Year,Dividends
0,2800.HK,2021,0.67
1,2800.HK,2022,0.64
2,2800.HK,2023,0.65
3,2800.HK,2024,0.77


In [6]:
df = pd.merge(df, df_div, on=['Ticker', 'Year'], how='inner')
df

,Ticker,Year,First_Close,Last_Close,Difference,Dividends
0,2800.HK,2021,23.82997,20.79711,-3.03286,0.67
1,2800.HK,2022,20.69100,18.32262,-2.36838,0.64
2,2800.HK,2023,18.63535,16.38940,-2.24595,0.65
3,2800.HK,2024,16.15105,20.04050,3.88945,0.77


In [7]:
df['Growth (%)'] = (df['Difference']+df['Dividends'])/df['First_Close']*100
df['Growth (%)'] = df['Growth (%)'].round(2) 
df

,Ticker,Year,First_Close,Last_Close,Difference,Dividends,Growth (%)
0,2800.HK,2021,23.82997,20.79711,-3.03286,0.67,-9.92
1,2800.HK,2022,20.69100,18.32262,-2.36838,0.64,-8.35
2,2800.HK,2023,18.63535,16.38940,-2.24595,0.65,-8.56
3,2800.HK,2024,16.15105,20.04050,3.88945,0.77,28.85


In [8]:
df.to_csv('TraHK.csv', index=False)